In [1]:
import pandas as pd
from sklearn.impute import SimpleImputer


In [2]:
# Load dataset
df = pd.read_csv("Telco_Customer_Churn_Dataset.csv")

# Drop customerID
if 'customerID' in df.columns:
    df.drop(columns=['customerID'], inplace=True)

# Convert TotalCharges to numeric
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# Handle missing values
num_cols = df.select_dtypes(include=['int64','float64']).columns
cat_cols = df.select_dtypes(include=['object']).columns.drop('Churn')

df[num_cols] = SimpleImputer(strategy='median').fit_transform(df[num_cols])
df[cat_cols] = SimpleImputer(strategy='most_frequent').fit_transform(df[cat_cols])

# Encode target column
df['ChurnFlag'] = df['Churn'].map({'Yes':1,'No':0})

print("✅ Dataset ready for segmentation")


✅ Dataset ready for segmentation


In [3]:
df['tenure_bucket'] = pd.cut(
    df['tenure'], 
    bins=[-1,3,12,24,48,72], 
    labels=['0-3','4-12','13-24','25-48','49-72']
)
print("✅ Tenure buckets created")


✅ Tenure buckets created


In [4]:
df['monthly_bucket'] = pd.cut(df['MonthlyCharges'], bins=4)
print("✅ Monthly charge buckets created")


✅ Monthly charge buckets created


In [5]:
seg_summary = (
    df.groupby(['tenure_bucket','monthly_bucket'])['ChurnFlag']
    .mean()
    .reset_index()
)
seg_summary.rename(columns={'ChurnFlag':'ChurnRate'}, inplace=True)
print("✅ Segmentation summary created")


✅ Segmentation summary created


/var/folders/8g/lxw7k2fx0cncl3vqj3wm1h4c0000gn/T/ipykernel_1792/292527878.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(['tenure_bucket','monthly_bucket'])['ChurnFlag']


In [6]:
print("Churn Rate by Segments:")
print(seg_summary)


Churn Rate by Segments:
   tenure_bucket    monthly_bucket  ChurnRate
0            0-3   (18.15, 43.375]   0.371508
1            0-3    (43.375, 68.5]   0.507812
2            0-3    (68.5, 93.625]   0.738155
3            0-3  (93.625, 118.75]   0.808511
4           4-12   (18.15, 43.375]   0.142857
5           4-12    (43.375, 68.5]   0.263359
6           4-12    (68.5, 93.625]   0.564039
7           4-12  (93.625, 118.75]   0.783333
8          13-24   (18.15, 43.375]   0.077703
9          13-24    (43.375, 68.5]   0.210762
10         13-24    (68.5, 93.625]   0.402235
11         13-24  (93.625, 118.75]   0.544218
12         25-48   (18.15, 43.375]   0.051402
13         25-48    (43.375, 68.5]   0.102990
14         25-48    (68.5, 93.625]   0.268145
15         25-48  (93.625, 118.75]   0.376694
16         49-72   (18.15, 43.375]   0.016632
17         49-72    (43.375, 68.5]   0.056716
18         49-72    (68.5, 93.625]   0.065183
19         49-72  (93.625, 118.75]   0.182620
